# Pandas Finance

Obtendo um hitórico do estoque de preços do Yahoo Finance.

In [ ]:
# Vamos carregar as bibliotecas necessárias
import pandas_datareader as pdr
import datetime
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
# Vamos definir um período
inicio = datetime.datetime(2017, 1, 1)
fim = datetime.datetime(2017,11, 22)
f = pdr.DataReader("F", 'yahoo', inicio, fim)

In [ ]:
f.loc['2017-01-04']

Hitórico das ações (Dividendos e divisões de ações ) com dados do Yahoo Finança:

In [ ]:
inicio = datetime.datetime(2008, 1, 1)
fim = datetime.datetime(2017,11, 22)
petro=pdr.DataReader('PETR4.SA', 'yahoo-actions', inicio, fim)
petro

In [ ]:
petro.loc['2009-07-06']

In [ ]:
inicio = datetime.datetime(2010, 1, 1)
fim = datetime.datetime(2017, 1, 1)
f = pdr.DataReader("BAHI3.SA", 'yahoo-dividends', inicio, fim)
f

In [ ]:
inicio = datetime.datetime(2017, 1, 1)
fim = datetime.datetime(2017, 11, 22)
ibov=pdr.get_data_google('INDEXBVMF:IBOV',inicio,fim)
ibov

Um DataFrame é uma coleção de objetos e Séries. Fatiando o DataFrame retorna uma Sérei em colunas

In [ ]:
ts=ibov['Close'][-10:]
ts

In [ ]:
type(ts)

In [ ]:
data=ts.index[5]
data

In [ ]:
ts[data]

In [ ]:
ts[5]

Também podemos selecionar multiplas colunas

In [ ]:
ibov[['Open', 'Close']].head()

Novas colunas podem ser adicionadas enquanto se roda

In [ ]:
ibov['dif'] = ibov.Open - ibov.Close
ibov.head()

Também podemos deletar enquanto usamos

In [ ]:
del ibov['dif']
ibov.head()

Também podemos usar uma outra fonte como o google finances

In [ ]:
start = datetime.datetime(2017, 10, 1)
end = datetime.datetime(2017, 11, 22)
vale_dados = pdr.get_data_google('NYSE:VALE',start,end)
vale_dados

In [ ]:
start = datetime.datetime(2013, 10, 1)
end = datetime.datetime(2017, 11, 22)
bahia = pdr.get_data_yahoo('BAHI3.SA',start,end)
bahia

## Cálculos Financeiros comuns

- Movimentação de médias
- Retornos

In [ ]:
fech_ba = bahia['Adj Close']

In [ ]:
med_ba = pd.rolling_mean(fech_ba, 40)
med_ba[10:]

Considere o Retorno definido como:

$$r_t = \frac{p_t - p_{t-1}}{p_{t-1}} = \frac{p_t}{p_{t-1}} - 1$$

In [ ]:
rets = fech_ba / fech_ba.shift(1) -1
rets.head()

Alternativamente poderia ser .pct_change()

In [ ]:
fech_ba.pct_change().head()

## Gráficos básicos

In [ ]:
%matplotlib inline

In [ ]:
fech_ba.plot(label='BAHIA3.SA')
med_ba.plot(label='média')
plt.legend()

## Outros ativos

In [ ]:
start = datetime.datetime(2017, 1, 1)
end = datetime.datetime(2017, 11, 22)
petro=pdr.get_data_yahoo('PETR4.SA',start,end)
petro

In [ ]:
#Vamos gravar os resultado anteriores em um arqivo csv
petro.to_csv('petro.csv')
!head petro.csv

In [ ]:
# Define as ações para download. Vamos usar  'BAHI3.SA', 'PETR4.SA', 'VALE5.SA' 
acoes = ["BAHI3.SA", "PETR4.SA", "VALE5.SA"]

# Define a fonte online a ser usada
data_fonte = 'yahoo'

# Vamos usar os dados disponíveis entre 01/01/2010 e 22/11/2017.
data_inicial = '2010-01-01'
data_final = '2017-11-22'

# Use o pandas_reader.data.DataReader para carregar os dados desejados. Faça
painel_dados = pdr.DataReader(acoes, data_fonte, data_inicial, data_final)

# Obtendo apenas os preços de fechamento ajustados. Isso retornará um Pandas DataFrame
# O índice neste DataFrame é o principal índice do painel_dados.
fechado = painel_dados.loc['Close']

# Obtendo todos os dias da semana entre entre 01/01/2010 e 22/11/2017.
todos_dias = pd.date_range(start=data_inicial, end=data_final, freq='B')


# Como alinhamos os preços existentes em adj_close com o nosso novo conjunto de datas?
# Tudo o que precisamos fazer é reindex fechar usando todos_diass como o novo índice

fechado = fechado.reindex(todos_dias)

fechado.head(10)

Como o painel_dados se parece? data.DataReader retorna um objeto de painel, que pode ser considerado como uma matriz 3D. A primeira dimensão consiste nos vários campos que o Yahoo Finance retorna para um determinado instrumento, ou seja, os preços Open, High, Low, Close e Adj Close para cada data. A segunda dimensão contém as datas. O terceiro contém os identificadores do instrumento.

Vamos ver o que panel_data realmente é:

In [ ]:
painel_dados

# Preparando os Dados

Considere que estamos interessados em trabalhar com os preços da `Adj Close` que já foram ajustados pelo financiamento do Yahoo para contabilizar ações corporativas, como dividendos e divisões de ações. Queremos garantir que todos os dias da semana estejam incluídos em nosso conjunto de dados, o que muitas vezes é desejável para estratégias de negociação quantitativas. Claro, alguns dias da semana podem ser feriados públicos, caso em que nenhum preço estará disponível. Por esse motivo, iremos preencher os preços que faltam com os preços disponíveis mais recentes. Tudo isso é, novamente, muito fácil com os pandas:

In [ ]:
# Obtendo apenas os preços de fechamento ajustados. Isso retornará um Pandas DataFrame
# O índice neste DataFrame é o índice principal do panel_data.
adj_close = painel_dados.loc['Adj Close']

# Obtendo todos os dias da semana entre entre 01/01/
2010 e 22/11/2017.
todos_dias = pd.date_range(start=data_inicial, end=data_final, freq='B')

# Como alinhamos os preços existentes em adj_close com o nosso novo conjunto de datas?
# Tudo o que precisamos fazer é reindex adj_close usando all_weekdays como o novo índice
adj_close = adj_close.reindex(todos_dias)


# Reindexing irá inserir valores faltantes (NaN) para as datas que não estavam presentes
# no conjunto original. Para lidar com isso, podemos preencher as faltas, substituindo-as
# com o último preço disponível para cada instrumento.
adj_close = adj_close.fillna(method='ffill')

Inicialmente, adj_close contém todos os preços de fechamento ajustados para todos os instrumentos e todas as datas que o Yahoo retornou. Alguns dias da semana podem estar faltando nos dados fornecidos pelo Yahoo. Por esse motivo, criamos uma série de todos os dias da semana entre a primeira e a última data de interesse e os armazenamos na variável all_weekdays. Obter todos os dias da semana é conseguido passando o parâmetro chamado freq = 'B' para a função pd.date_range (). Esta função retorna um DatetimeIndex que é mostrado abaixo:

In [ ]:
todos_dias

Alinhar o DataFrame original com o novo DatetimeIndex é realizado por substituição do DatetimeIndex inicial do adj_close DataFrame. Se alguma das novas datas não foi incluída no original DatetimeIndex, os preços dessa data serão preenchidos com NaNs. Por este motivo, iremos preencher todos os Nacionais resultantes com o último preço disponível. O DataFrame final e limpo é mostrado abaixo:

In [ ]:
adj_close.head(7)

## Olhando para os Dados

Nosso conjunto de dados está agora completo e sem valores faltantes. Podemos ver um resumo dos valores em cada um dos instrumentos ao chamar o método `describe()` de um Pandas DataFrame:

In [ ]:
adj_close.describe()

Suponhamos que gostaríamos de traçar a série temporal VALE5.SA. Também gostaríamos de ver como o estoque se comporta em comparação com uma média móvel de curto e longo prazo de seu preço. Uma média móvel simples da série temporal original é calculada tomando por cada data a média dos últimos preços de W (incluindo o preço na data de juros). A pandas tem a função `rolling()`, uma função interna para Séries que retorna um objeto rolando para uma janela definida pelo usuário, e. 20 dias. Uma vez que um objeto de rolamento foi obtido, várias funções podem ser aplicadas sobre ele, como `sum()`, `std()` (para calcular o desvio padrão dos valores na janela) ou `mean()`. Ver abaixo:

In [ ]:
%matplotlib inline

In [ ]:
# Obtém a série temporal da VALE5.SA. Essa agora returna umv objeto  Serie Pandas indexado pela data.
vale = adj_close.loc[:, 'VALE5.SA']
# Calcule os 20 e 100 dias das médias móveis dos preços de fechamento
rolamento_curto_vale = vale.rolling(window=20).mean()
rolamento_longo_vale = vale.rolling(window=100).mean()


# Faz um gráfico de tudo 
fig = plt.figure()
ax = fig.add_subplot(1,1,1)
ax.plot(vale.index, vale, label='VALE%.SA')
ax.plot(rolamento_curto_vale.index, rolamento_curto_vale, label='Janela de 20 dias')
ax.plot(rolamento_longo_vale.index, rolamento_longo_vale, label='Janela de 100 dias')
ax.set_xlabel('Data')
ax.set_ylabel('Ajuste do preço de encerramento ($)')
ax.legend()